In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
# Print the first 3, just so you can see what they look like.
print(ngrams[:3])

[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]


In [3]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [4]:
print(word_to_ix)

{'And': 0, 'cold.': 1, 'sunken': 2, 'deep': 3, 'warm': 4, 'Were': 5, 'dig': 6, 'and': 7, 'asked,': 8, 'an': 9, 'thy': 10, "youth's": 11, 'own': 12, 'mine': 13, 'where': 14, 'shame,': 15, 'in': 16, 'When': 17, 'beauty': 18, 'days;': 19, 'eyes,': 20, "feel'st": 21, 'made': 22, 'winters': 23, 'small': 24, 'How': 25, 'Shall': 26, 'livery': 27, 'lusty': 28, "totter'd": 29, 'a': 30, 'lies,': 31, 'trenches': 32, 'sum': 33, 'If': 34, 'fair': 35, 'gazed': 36, 'weed': 37, 'thou': 38, 'now,': 39, 'on': 40, 'use,': 41, 'thine': 42, 'held:': 43, 'when': 44, 'old': 45, 'by': 46, 'see': 47, 'Where': 48, 'to': 49, "'This": 50, 'it': 51, 'To': 52, 'thriftless': 53, 'be': 54, 'within': 55, 'besiege': 56, 'treasure': 57, 'being': 58, 'art': 59, 'make': 60, 'This': 61, 'praise': 62, 'answer': 63, 'of': 64, 'child': 65, 'new': 66, 'much': 67, "deserv'd": 68, 'field,': 69, 'brow,': 70, 'shall': 71, 'more': 72, 'say,': 73, "beauty's": 74, 'worth': 75, 'all-eating': 76, 'my': 77, 'all': 78, 'couldst': 79, 'Pr

In [5]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        print(embeds.shape)
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [6]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [7]:
for epoch in range(2):
    total_loss = 0
    for context, target in ngrams:

        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        model.zero_grad()

        log_probs = model(context_idxs)

        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    losses.append(total_loss)
print(losses) 

torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
torch.Size([1, 20])
